In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:

import os
import sys
from dotenv import load_dotenv
import pandas as pd
from pgvector.psycopg import register_vector
import json
import pyproj
import tqdm
pyproj.datadir.set_data_dir('/Users/jm/Code/mapgpt-mvp/.conda/share/proj')

sys.path.append('../src')

from infrastructure.gpt4 import GPT4, generate_openai_embedding_request
from infrastructure.postgres_db import PostgresDB
from application.maps.choropleth_map import ChoroplethMap
import plotly.graph_objects as go

load_dotenv()

gpt4 = GPT4()

db_name = os.environ.get("DB_NAME")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")
db = PostgresDB(db_name=db_name, db_user=db_user, db_password=db_password)


In [35]:
table = "comuna"
column = "nombre"

values = pd.read_sql(f"SELECT DISTINCT {column} FROM {table}", con=db.engine)[column]
requests = [generate_openai_embedding_request(i, text) for i, text in enumerate(values)]
len(requests)


346

In [36]:
batch_id = gpt4.send_batch_embedding(requests, f"batch_embedding__{table}_{column}.jsonl")

In [49]:
gpt4.get_batch(batch_id).status

'completed'

In [50]:
results = gpt4.get_batch_result(gpt4.get_batch(batch_id).output_file_id)
requests_df = pd.DataFrame([{"custom_id": d["custom_id"], "input": d["body"]["input"]} for d in requests]).set_index("custom_id")
responses_df = pd.DataFrame([{"custom_id": d["custom_id"], "embedding": d["response"]["body"]["data"][0]["embedding"]} for d in results]).set_index("custom_id")
input_embedding = requests_df.join(responses_df).set_index("input")["embedding"].dropna().to_dict()

In [51]:
db.create_connection()
register_vector(db.conn)

with db.create_connection() as conn:
    with conn.cursor() as cur:
        with cur.copy(f"COPY embeddings_openai_small (table_name, column_name, text, embedding) FROM STDIN WITH (FORMAT BINARY)") as copy:
            for name, embedding in tqdm.tqdm(input_embedding.items(), total=len(input_embedding)):
                # formatted_embedding = [round(num, 6) for num in embedding]
                embedding_str = json.dumps(embedding, separators=(',', ':'))
                copy.set_types(["text", "text", "text", "vector"])
                copy.write_row([table, column, name, embedding])
    conn.commit()


100%|██████████| 346/346 [00:00<00:00, 2415.46it/s]


To update tables with embedding values:
```sql
UPDATE table_name co
SET nombre__emb_openai_small = e.embedding
FROM embeddings_openai_small e
WHERE e."text" = co.nombre```